# 4.1 ***Votre Modèle***
___
Bravo, vous êtes arrivé jusqu'ici ! Vous comprenez maintenant les concepts clés des réseaux de neurones et leur entraînement, mais vous n'en avez pas encore créé un...
Pas de panique, cette tâche finale vous guidera dans la création d'un réseau de neurones entraîné pour détecter n'importe quel chiffre écrit à la main sur une image de 28 par 28 pixels !


In [ ]:
# importons toutes les librairies nécéssaires:)

import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
import time

Tout d'abord, nous devons préparer le jeu de données : nous tensorions et normalisons le jeu de données pour simplifier le traitement. *(Vous n'avez pas besoin de comprendre cela pour l'instant, chargez simplement le jeu de données, mais n'hésitez pas à poser des questions !)*


In [ ]:
# Exécutez simplement ce code

# transformation et normalisation des données
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# chargement du jeu de données
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# affichage des informations
print(f"Len train dataset : {len(train_dataset)}")
print(f"Len test  dataset : {len(test_dataset)}")


Pour comprendre ce que contient ce code, vous pouvez essayer le code ci-dessous pour visualiser certains exemples !

***N'hésitez pas à modifier la variable NUMBER_OF_ELEMENTS pour voir plusieurs exemples ou aucun***


In [ ]:
# Visualisation de certains éléments du jeu de données

# TODO : Changer le nombre d'éléments à afficher
NUMBER_OF_ELEMENTS = ...

def imshow(img):
    img = img * 0.5 + 0.5  # Dénormalisation
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)), cmap='gray')
    plt.axis('off')
    plt.show()

train_loader_vis = torch.utils.data.DataLoader(train_dataset, batch_size=NUMBER_OF_ELEMENTS, shuffle=True)

# Image aléatoire
dataiter = iter(train_loader_vis)
images, labels = next(dataiter)

imshow(torchvision.utils.make_grid(images))
print('Labels :', ' '.join(f'{labels[j].item()};' for j in range(NUMBER_OF_ELEMENTS)))


___
### Créez votre modèle !

Voici le gros du travail... Créez votre propre modèle **à partir de zéro** !!

***N'hésitez pas à commencer simplement la première fois, puis à essayer d'implémenter une architecture plus complexe. Pensez d'abord à une couche entièrement connectée qui commence par une couche de flattening, et recréez un modèle plus complexe ensuite !***

Pour comprendre comment fonctionnent les réseaux de neurones convolutifs (CNN) ou entièrement connectés (FCN, modèles linéaires), allez consulter la partie bonus [ici](<../Partie I - Propagation Avant /1.1 concept_du_reseau_de_neurones.ipynb>).

Nous allons essayer de vous aider à créer ce modèle :


In [ ]:
# TODO: Définir le taux d'apprentissage
LEARNING_RATE = ...

class MNISTModel(nn.Module):
    def __init__(self):
        super(MNISTModel, self).__init__()

        self.flatten = nn.Flatten()
        self.fc1 = ... # 28*28 -> ce que vous souhaitez
        self.fc2 = ...
        ...
        self.fc = ... # ce que vous souhaitez -> 10

        self.loss = ...
        self.optimizer = ... # utilisez les paramètres self et le taux d'apprentissage
        self.relu = ...

        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.to(self.device)

    def forward(self, x):
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = ...
        ...
        return x

    def train_model(self, epochs, train_loader):
        self.train()  # Mode d'entraînement

        for epoch in range(epochs):
            start_time = time.time()  # Temps de début de l'époque
            running_loss = 0.0
            total_batches = 0

            for i, data in enumerate(train_loader): # Énumère les données, tout le jeu de données
                inputs, labels = data
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                # TODO: Implémenter la boucle d'entraînement
                
                # Gradient mis à zéro
                # Passage avant
                # Calcul de la perte
                # Passage arrière
                # Étape d'optimisation

                running_loss += loss.item()
                total_batches += 1  # juste pour aider à l'affichage
                # afficher tous les 8 mini-lots
                if (i + 1) % 8 == 0 or (i + 1) == len(train_loader):
                    print(f"\rÉpoque {epoch + 1}/{epochs} | Lot {i + 1}/{len(train_loader)} | Perte : {loss.item():.4f}", end='')

            avg_loss = running_loss / len(train_loader)
            epoch_time = time.time() - start_time

            print(f"\nÉpoque {epoch + 1}/{epochs} terminée | Perte Moyenne : {avg_loss:.4f} | Temps : {epoch_time:.2f} secondes")

        # changez le chemin model_path si nécessaire
        model_path = "mnist_model.pth"
        print('Entraînement terminé, sauvegarde du modèle à :', model_path)
        torch.save(self.state_dict(), model_path)

    def test_model(self, test_loader):
        self.eval()  # Mode d'évaluation
        correct = 0
        total = 0
        with torch.no_grad():
            for data in test_loader:
                images, labels = data
                images, labels = images.to(self.device), labels.to(self.device)
                outputs = self(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(f'Précision du modèle sur {total} images : {100 * correct / total:.2f}%')

    def load_weights(self, model_path):
        self.load_state_dict(torch.load(model_path))


### Initialiser le Modèle

Initialisez le modèle en dehors de la boucle d'entraînement pour ne le charger qu'une seule fois. Si vous souhaitez recommencer l'entraînement avec des poids aléatoires, vous pouvez redémarrer cette cellule. Sinon, l'entraînement continuera à partir de la **`dernière valeur de perte`**.


In [ ]:
model = MNISTModel()

### Prêt à Entraîner Votre Modèle !

Si vous êtes arrivé jusqu'ici, vous êtes maintenant prêt à entraîner votre modèle. Mais avant de commencer, n'oubliez pas de configurer votre **BATCH_SIZE** et vos **EPOCHS**. Clarifions ce que ces termes signifient :

- *Époques* (*Epochs*) représentent le nombre de fois que votre ensemble de données complet sera traité par le modèle pendant l'entraînement. Par exemple, avec un jeu de données de 60 000 images, définir ***EPOCHS=10*** signifie que ces 60 000 images seront passées dans le modèle 10 fois au total, permettant au modèle d'ajuster ses poids à chaque passage.

- *Taille de lot* (*Batch size*) est le nombre d'images traitées à chaque passage avant de calculer la perte et d'appliquer la rétropropagation pour mettre à jour les poids du modèle. Par exemple, avec ***BATCH_SIZE=64***, 64 images sont traitées simultanément par le modèle. Après avoir calculé la perte pour ce lot, la rétropropagation ajuste les poids en fonction des résultats de ce lot.

Pour chaque époque, votre modèle traitera plusieurs *lots* (*batches*), chacun contenant un nombre spécifié d'images (`batch_size`). Le nombre total de lots par époque peut être calculé avec la formule suivante :

$$
{\text{LOT}} = \frac{\text{len\_dataset}}{\text{batch\_size}}
$$


In [ ]:
# Charge les jeux de données avec une taille de lot (batch_size)
BATCH_SIZE = ...

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


EPOCHS = ...  # Nombre d'époques

# Entraîner le modèle
model.train_model(EPOCHS, train_loader)


Il est maintenant temps de jouer avec les autres étudiants ! Essayez de comparer qui obtient la meilleure précision ! Ce sera le ***meilleur modèle*** :)

*(essayez d'atteindre les 99% de précision)*


In [ ]:
model.test_model(test_loader)